# Parameter Management

Once we have chosen an architecture
and set our hyperparameters,
we proceed to the training loop,
where our goal is to find parameter values
that minimize our objective function. 
After training, we will need these parameters 
in order to make future predictions.
Additionally, we will sometimes wish 
to extract the parameters 
either to reuse them in some other context,
to save our model to disk so that 
it may be exectuted in other software,
or for examination in the hopes of 
gaining scientific understanding.

Most of the time, we will be able 
to ignore the nitty-gritty details
of how parameters are declared
and manipulated, relying on DJL
to do the heavy lifting.
However, when we move away from 
stacked architectures with standard layers, 
we will sometimes need to get into the weeds
of declaring and manipulating parameters. 
In this section, we cover the following:

* Accessing parameters for debugging, diagnostics, and visualiziations.
* Parameter initialization.
* Sharing parameters across different model components.

We start by focusing on an MLP with one hidden layer.

In [ ]:
%mavenRepo snapshots https://oss.sonatype.org/content/repositories/snapshots/

%maven ai.djl:api:0.7.0-SNAPSHOT
%maven ai.djl:model-zoo:0.7.0-SNAPSHOT
%maven org.slf4j:slf4j-api:1.7.26
%maven org.slf4j:slf4j-simple:1.7.26
    
%maven net.java.dev.jna:jna:5.3.0
%maven ai.djl.mxnet:mxnet-engine:0.7.0-SNAPSHOT
%maven ai.djl.mxnet:mxnet-native-auto:1.7.0-a

In [ ]:
import ai.djl.*;
import ai.djl.ndarray.*;
import ai.djl.ndarray.types.*;
import ai.djl.ndarray.index.*;
import ai.djl.nn.*;
import ai.djl.nn.core.*;
import ai.djl.training.*;
import ai.djl.training.initializer.*;
import ai.djl.training.dataset.*;
import ai.djl.util.*;
import ai.djl.translate.*;
import ai.djl.inference.Predictor;

In [ ]:
NDManager manager = NDManager.newBaseManager();

NDArray x = manager.randomUniform(0, 1, new Shape(2, 4));

Model model = Model.newInstance("lin-reg");

SequentialBlock net = new SequentialBlock();

net.add(Linear.builder().setUnits(8).build());
net.add(Activation.reluBlock());
net.add(Linear.builder().setUnits(1).build());
net.setInitializer(new NormalInitializer());
net.initialize(manager, DataType.FLOAT32, x.getShape());

model.setBlock(net);

Predictor<NDList, NDList> predictor = model.newPredictor(new NoopTranslator());

predictor.predict(new NDList(x)).singletonOrThrow(); // forward computation

## Parameter Access

Let us start with how to access parameters
from the models that you already know.
When a model is defined via the SequentialBlock class,
we can first access any layer by indexing 
into the model as though it were a list.
Each layer's parameters are conveniently 
returned with a call to the `getParameters()` method. 
We can inspect the parameters of the `net` defined above.

In [ ]:
ParameterList params = net.getParameters();
NDArray dense0Weight = params.get(0).getValue().getArray();
NDArray dense0Bias = params.get(1).getValue().getArray();

NDArray dense1Weight = params.get(2).getValue().getArray();
NDArray dense1Bias = params.get(3).getValue().getArray();

System.out.println(dense0Weight);
System.out.println(dense0Bias);

System.out.println(dense1Weight);
System.out.println(dense1Bias);

The output tells us a few important things.
First, each fully-connected layer 
has two parameters, e.g., 
`dense0Weight` and `dense0Bias`,
corresponding to that layer's 
weights and biases, respectively.
The `parametersList` variable is a `ParameterList` which holds `Pair`s which contain
key-value pairs of the layer name and a parameter which is repesented as an instance of the 
`Parameter` class.
We can get the `Parameter` from a `Pair` by calling `getValue()` on it.
With `Parameter`, we can then get the underlying numerical values as `NDArray`s by calling 
`getArray()` on them!
Both the weights and biases are stored as single precision floats(`FLOAT32`).


### Targeted Parameters

Parameters are complex objects,
containing data, gradients,
and additional information.
That's why we need to request the data explicitly.
Note that the bias vector consists of zeroes
because we have not updated the network
since it was initialized.

Note that unlike the biases, the weights are nonzero. 
This is because unlike biases, 
weights are initialized randomly. 
In addition to `getArray()`, each `Parameter`
also provides a `requireGradient()` method which
returns whether the parameter needs gradients to be computed
(which we set on the `NDArray` with `attachGradient()`).
The gradient has the same shape as the weight. 
To actually access the gradient, we simply call `getGradient()` on the
`NDArray`.
Because we have not invoked backpropagation 
for this network yet, its values are all 0.
We would invoke it by creating a `GradientCollector` instance and
run our calculations inside it.

In [ ]:
dense0Weight.getGradient();

### Collecting Parameters from Nested Blocks

Let us see how the parameter naming conventions work 
if we nest multiple blocks inside each other. 
For that we first define a function that produces Blocks 
(a Block factory, so to speak) and then 
combine these inside yet larger Blocks.

In [ ]:
public SequentialBlock block1() {
    SequentialBlock net = new SequentialBlock();
    net.add(Linear.builder().setUnits(32).build());
    net.add(Activation.reluBlock());
    net.add(Linear.builder().setUnits(16).build());
    net.add(Activation.reluBlock());
    return net;
}

public SequentialBlock block2() {
    SequentialBlock net = new SequentialBlock();
    for (int i = 0; i < 4; i++) {
        net.add(block1());
    }
    return net;
}

SequentialBlock rgnet = new SequentialBlock();
rgnet.add(block2());
rgnet.add(Linear.builder().setUnits(10).build());
rgnet.setInitializer(new NormalInitializer());
rgnet.initialize(manager, DataType.FLOAT32, x.getShape());

Model model = Model.newInstance("rgnet");
model.setBlock(rgnet);

Predictor<NDList, NDList> predictor = model.newPredictor(new NoopTranslator());

predictor.predict(new NDList(x)).singletonOrThrow();

Now that we have designed the network, 
let us see how it is organized.
We can get the list of named parameters by calling `getParameters()`.
However, we not only want to see the parameters, but also how
our network is structured.
To see our network architecture, we can simply print out the block whose architecture we want to see.

In [ ]:
/* Network Architecture for RgNet */
rgnet

In [ ]:
/* Parameters for RgNet */
for (var param : rgnet.getParameters()) {
    System.out.println(param.getValue().getArray());
}

Since the layers are hierarchically nested,
we can also access them by calling their `getChildren()` method
to get a list of their inner blocks, then calling `get(i)` to get the
block at the index `i` we want, and then finally `getValue()` to get the actual
block. Then we have to repeat that to get that blocks child and so on.
This is a bit clunky and soon there'll be an easier method to do this, but for now
this is the best method.

Here, we access the first major block, 
within it the second subblock, 
and within that the bias of the first layer,
with as follows:

In [ ]:
Block majorBlock1 = rgnet.getChildren().get(0).getValue();
Block subBlock2 = majorBlock1.getChildren().get(1).getValue();
Block linearLayer1 = subBlock2.getChildren().get(0).getValue();
NDArray bias = linearLayer1.getParameters().get(1).getValue().getArray();
bias

## Parameter Initialization

Now that we know how to access the parameters,
let us look at how to initialize them properly.
We discussed the need for initialization in :numref:`sec_numerical_stability`. 
By default, DJL initializes weight matrices
based on your set initializer 
and the bias parameters are all set to $0$.
However, we will often want to initialize our weights
according to various other protocols. 
DJL's `ai.djl.training.initializer` package provides a variety 
of preset initialization methods.
If we want to create a custom initializer,
we need to do some extra work.

### Built-in Initialization

In DJL, when setting the initializer for blocks, the default `setInitializer()` function does not overwrite
any previous set initializers. So if you set an initializer earlier, but decide you want to change your initializer and call `setInitializer()` again, the second `setInitializer()` will NOT overwrite your first one.

Additionally, when you call `setInitializer()` on a block, all internal blocks will also call `setInitializer()` with the same given `initializer`.

This means that we can call `setInitializer()` on the highest level of a block and know that all internal blocks that do not have an initializer already set will be set to that given `initializer`.

This setup has the advantage that we don't have to worry about our `setInitializer()` overriding our previous `initializer`s on internal blocks, but also means that we cannot change our `initializer` without digging deeper into the code.

If you want to however, you can explicitly set an initializer for a `Parameter` by calling its `setInitializer()` function directly and passing in `true` to the overwrite input.

Let us begin by calling on built-in initializers. 
The code below initializes all parameters 
to a given constant value 1, 
by using the `ConstantInitializer()` initializer. 

Note that this will not do anything currently since we have already set
our initializer in the previous code block.
We can verify this by checking the weight of a parameter.

In [ ]:
net.setInitializer(new ConstantInitializer(1));
net.initialize(manager, DataType.FLOAT32, x.getShape());
Block linearLayer = net.getChildren().get(0).getValue();
NDArray weight = linearLayer.getParameters().get(0).getValue().getArray();
weight

We can see these initializations however if we create a new network.
Let us write a function to create these network architectures for us
conveniently.

In [ ]:
public SequentialBlock getNet() {
    SequentialBlock net = new SequentialBlock();
    net.add(Linear.builder().setUnits(8).build());
    net.add(Activation.reluBlock());
    net.add(Linear.builder().setUnits(1).build());
    return net;
}

If we run our previous initializer on this new net and check a parameter, we'll
see that everything is initialized properly! (to 7777!)

In [ ]:
SequentialBlock net = getNet();
net.setInitializer(new ConstantInitializer(7777));
net.initialize(manager, DataType.FLOAT32, x.getShape());
Block linearLayer = net.getChildren().get(0).getValue();
NDArray weight = linearLayer.getParameters().get(0).getValue().getArray();
weight

We can also initialize all parameters 
as Gaussian random variables 
with standard deviation $.01$.

In [ ]:
SequentialBlock net = getNet();
net.setInitializer(new NormalInitializer());
net.initialize(manager, DataType.FLOAT32, x.getShape());
Block linearLayer = net.getChildren().get(0).getValue();
NDArray weight = linearLayer.getParameters().get(0).getValue().getArray();
weight

We can also apply different initializers for certain Blocks.
For example, below we initialize the first layer
with the `Xavier` initializer
and initialize the second layer 
to a constant value of 0.

We will do this without the `getNet()` function as it will be easier
to have the reference to each block we want to set.


In [ ]:
SequentialBlock net = new SequentialBlock();
Linear linear1 = Linear.builder().setUnits(8).build();
net.add(linear1);
net.add(Activation.reluBlock());
Linear linear2 = Linear.builder().setUnits(1).build();
net.add(linear2);

linear1.setInitializer(new XavierInitializer());
linear1.initialize(manager, DataType.FLOAT32, x.getShape());

linear2.setInitializer(Initializer.ZEROS);
linear2.initialize(manager, DataType.FLOAT32, x.getShape());

System.out.println(linear1.getParameters().get(0).getValue().getArray());
System.out.println(linear2.getParameters().get(0).getValue().getArray());

### Custom Initialization

Sometimes, the initialization methods we need 
are not standard in DJL. 
In these cases, we can define a class to implement the `Initializer` interface. 
We only have to implement the `initialize()` function,
which takes an `NDManager`, a `Shape`, and the `DataType`. 
We then create the `NDArray` with the aforementioned `Shape` and `DataType`
and initialize it to what we want! You can also design your
initializer to take in some parameters. Simply declare them
as fields in the class and pass them in as inputs to the constructor!
In the example below, we define an initializer
for the following strange distribution:

$$
\begin{aligned}
    w \sim \begin{cases}
        U[5, 10] & \text{ with probability } \frac{1}{4} \\
            0    & \text{ with probability } \frac{1}{2} \\
        U[-10, -5] & \text{ with probability } \frac{1}{4}
    \end{cases}
\end{aligned}
$$

In [ ]:
class MyInit implements Initializer {

    public MyInit() {}

    @Override
    public NDArray initialize(NDManager manager, Shape shape, DataType dataType) {
        System.out.printf("Init %s\n", shape.toString());
        // Here we generate data points 
        // from a uniform distribution [-10, 10]
        NDArray data = manager.randomUniform(-10, 10, shape, dataType);
        // We keep the data points whose absolute value is >= 5
        // and set the others to 0.
        // This generates the distribution `w` shown above.
        NDArray absGte5 = data.abs().gte(5); // returns boolean NDArray where 
                                             // true indicates abs >= 5 and
                                             // false otherwise
        return data.mul(absGte5); // keeps true indices and sets false indices to 0.
                                  // special operation when multiplying a numerical
                                  // NDArray with a boolean NDArray
    }

}

In [ ]:
SequentialBlock net = getNet();
net.setInitializer(new MyInit());
net.initialize(manager, DataType.FLOAT32, x.getShape());
Block linearLayer = net.getChildren().get(0).getValue();
NDArray weight = linearLayer.getParameters().get(0).getValue().getArray();
weight

Note that we always have the option 
of setting parameters directly by calling `getValue().getArray()` 
to access the underlying `NDArray`. 
A note for advanced users: 
you cannot directly modify parameters within a `GarbageCollector` scope.
You must modify them outside the `GarbageCollector` scope to avoid confusing 
the automatic differentiation mechanics.

In [ ]:
// '__'i() is an inplace operation to modify the original NDArray
NDArray weightLayer = net.getChildren().get(0).getValue()
    .getParameters().get(0).getValue().getArray();
weightLayer.addi(7);
weightLayer.divi(9);
weightLayer.set(new NDIndex(0, 0), 2020); // set the (0, 0) index to 2020
weightLayer;

## Tied Parameters

Often, we want to share parameters across multiple layers.
Later we will see that when learning word embeddings,
it might be sensible to use the same parameters
both for encoding and decoding words. 
We discussed one such case when we introduced :numref:`sec_model_construction`. 
Let us see how to do this a bit more elegantly. 
In the following we allocate a dense layer 
and then use its parameters specifically 
to set those of another layer.

In [ ]:
SequentialBlock net = new SequentialBlock();

// We need to give the shared layer a name 
// such that we can reference its parameters
Block shared = Linear.builder().setUnits(8).build();
SequentialBlock sharedRelu = new SequentialBlock();
sharedRelu.add(shared);
sharedRelu.add(Activation.reluBlock());

net.add(Linear.builder().setUnits(8).build());
net.add(Activation.reluBlock());
net.add(sharedRelu);
net.add(sharedRelu);
net.add(Linear.builder().setUnits(10).build());

NDArray x = manager.randomUniform(-10f, 10f, new Shape(2, 20), DataType.FLOAT32);

net.setInitializer(new NormalInitializer());
net.initialize(manager, DataType.FLOAT32, x.getShape());

model.setBlock(net);

Predictor<NDList, NDList> predictor = model.newPredictor(new NoopTranslator());
System.out.println(predictor.predict(new NDList(x)).singletonOrThrow());

// Check that the parameters are the same
NDArray shared1 = net.getChildren().get(2).getValue()
    .getParameters().get(0).getValue().getArray();
NDArray shared2 = net.getChildren().get(3).getValue()
    .getParameters().get(0).getValue().getArray();
shared1.eq(shared2);

This example shows that the parameters 
of the second and third layer are tied. 
They are not just equal, they are 
represented by the same exact `NDArray`. 
Thus, if we change one of the parameters,
the other one changes, too. 
You might wonder, 
*when parameters are tied
what happens to the gradients?*
Since the model parameters contain gradients,
the gradients of the second hidden layer
and the third hidden layer are added together
in `shared.getGradient()` during backpropagation.

## Summary

* We have several ways to access, initialize, and tie model parameters.
* We can use custom initialization.
* DJL has a sophisticated mechanism for accessing parameters in a unique and hierarchical manner.


## Exercises

1. Use the FancyMLP defined in :numref:`sec_model_construction` and access the parameters of the various layers.
1. Look at the [DJL documentation](https://javadoc.io/doc/ai.djl/api/latest/ai/djl/training/initializer/Initializer.html) and explore different initializers.
1. Try accessing the model parameters after `net.initialize()` and before `predictor.predict(x)` to observe the shape of the model parameters. What changes? Why?
1. Construct a multilayer perceptron containing a shared parameter layer and train it. During the training process, observe the model parameters and gradients of each layer.
1. Why is sharing parameters a good idea?